In [77]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [78]:
df = pd.read_csv("/kaggle/input/flickr8k/captions.txt", sep=",")

In [79]:
df.head(3)

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set of stairs in an entry way .
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .


In [80]:
def process_caption(caption):
    words = caption.split()
    return ['<sos>'] + words + ['<eos>']

def remove_single_char_word(word_list):
    lst = []
    for word in word_list:
        if len(word)>1:
            lst.append(word)

    return lst

df['cleaned_caption'] = df['caption'].apply(lambda caption : ['<start>'] + [word.lower() if word.isalpha() else '' for word in caption.split(" ")] + ['<end>'])
df['cleaned_caption']  = df['cleaned_caption'].apply(lambda x : remove_single_char_word(x))
df['seq_len'] = df['cleaned_caption'].apply(lambda x : len(x))
df['seq_len'].max()
df['cleaned_caption'].head(3)

0    [<start>, child, in, pink, dress, is, climbing, up, set, of, stairs, in, an, entry, way, <end>]
1                                              [<start>, girl, going, into, wooden, building, <end>]
2                                  [<start>, little, girl, climbing, into, wooden, playhouse, <end>]
Name: cleaned_caption, dtype: object

In [76]:
def pad_words(words, target_length=33, pad_token='<pad>'):
    length = len(words)
    if length < target_length:
        return words + [pad_token] * (target_length - length)
    else:
        return words[:target_length]

df['cleaned_caption'] = df['cleaned_caption'].apply(pad_words)
df.head(3)

,image,caption,cleaned_caption,seq_len
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set of stairs in an entry way .,"[<start>, child, in, pink, dress, is, climbing, up, set, of, stairs, in, an, entry, way, <end>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>]",16
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .,"[<start>, girl, going, into, wooden, building, <end>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>]",7
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .,"[<start>, little, girl, climbing, into, wooden, playhouse, <end>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>]",8


In [ ]:
csv_file_path = "/kaggle/input/flickr8k/captions.txt"
images_path = "/kaggle/input/flickr8k/Images/"

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import torchvision.transforms as transforms

class Flickr8K(Dataset):
    def __init__(self, csv_file, images_path):
        self.df = pd.read_csv(path, sep=",")
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = images_path + self.df.iloc[idx, 0]
        image = Image.open(img_path).convert("RGB")
        target = self.df.iloc[idx, 1]  
        return image, target

dataset = Flickr8K(csv_file_path, images_path)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)